In [1]:
import os
import base64
import exifread
import datetime
import polars as pl
import plotly.express as px
import pendulum
from tqdm import tqdm
from dash import Dash, dcc, html, Input, Output, State
from dash_extensions import EventListener

from jupyter_dash import JupyterDash
import webbrowser
import time

In [2]:
def encode_image(image_path):
    with open(image_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()
        return f"data:image/png;base64,{encoded}"

In [3]:
# Sample data
df = pl.DataFrame({
    "x": [1, 2, 3],
    "y": [4, 5, 4],
    "t": [7, 8, 9],
    "image_urls": [
        "./photos/20241004_043129.jpg",
        "./photos/20241004_061911.jpg",
        "./photos/PXL_20240927_214531595.jpg",
    ]
})

df.head()

x,y,t,image_urls
i64,i64,i64,str
1,4,7,"""./photos/20241004_043129.jpg"""
2,5,8,"""./photos/20241004_061911.jpg"""
3,4,9,"""./photos/PXL_20240927_21453159…"


# Hover implementation

In [4]:
app = Dash(__name__)

fig = px.scatter(
    df,
    x="x",
    y="y",
    hover_data={"t": True},
    custom_data=["image_urls"],
)

fig.update_traces(hovertemplate="%{customdata[1]}")

# List all required components
app.layout = html.Div([
    dcc.Graph(id="figure", figure=fig, style={"height": "100vh", "width": "100vw"}),  # dcc = Dash Core Components
    html.Img(id="hover-image"),
])

In [5]:
hover_image_style = {
    "position": "absolute",
    "top": "50px",
    "right": "50px",
    "width": "200px",
    "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}


@app.callback(
    Output("hover-image", "src"),    # first output from the below function to hover-image (see app.layout)
    Output("hover-image", "style"),  # second output from the below function to hover-image (see app.layout)

    Input("figure", "hoverData"),    # input from the scatter-plot (see app.layout) to the below function
)
def display_hover_image(hoverData):
    if hoverData:
        image_url = hoverData["points"][0]["customdata"][0]
        return encode_image(image_url), hover_image_style
    return "", {"display": "none"}

In [6]:
app.run_server(port=8050, debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


# Click implementation

In [7]:
# app = Dash(__name__)

# fig = px.scatter(
#     df,
#     x="x",
#     y="y",
#     hover_data={"t": True},
#     custom_data=["image_urls"],
# )

# fig.update_traces(
#     hovertemplate="%{customdata[1]}"
# )

# app.layout = html.Div([
#     dcc.Graph(id="figure", figure=fig, style={"height": "100vh", "width": "100vw"}),
#     html.Img(id="click-image", style={"display": "block"}),
# ])

In [8]:
# click_image_style = {
#     "position": "absolute",
#     "top": "50%",
#     "left": "50%",
#     "transform": "translate(-50%, -50%)",  # Offset by half the image dimensions to truly center it
#     "width": "200px",
#     "zIndex": "10",
#     "border": "1px solid black",
#     "display": "block",
# }


# @app.callback(
#     Output("click-image", "src"),
#     Output("click-image", "style"),
#     Output("figure", "clickData"),

#     Input("figure", "clickData"),
#     Input("click-image", "n_clicks"),
#     State("click-image", "style"),
# )
# def toggle_click_image(clickData, n_clicks, current_style):
#     if n_clicks and current_style["display"] == "block":
#         return "", {"display": "none"}, None

#     if clickData:
#         image_url = clickData["points"][0]["customdata"][0]
#         return encode_image(image_url), click_image_style, None

#     return "", {"display": "none"}, None

In [9]:
# app.run_server(port=8050, debug=True, use_reloader=False)

# Click anywhere implementation

In [7]:
app = JupyterDash(__name__)

fig = px.scatter(
    df,
    x="x",
    y="y",
    hover_data={"t": True},
    custom_data=["image_urls"],
)

fig.update_traces(hovertemplate="%{customdata[1]}")

event_listener_js = {
    "event": "click",                 # Listen for click events
    "props": ["clientX", "clientY"],  # Capture the click's position on the screen
}

app.layout = html.Div([
    # Wrappers over components to listen for any clicks
    EventListener(
        dcc.Graph(id="figure", figure=fig, style={"height": "100vh", "width": "100vw"}),
        events=[event_listener_js],
        id="figure-clicks",
    ),
    EventListener(
        html.Img(id="click-image", style={"display": "block"}),
        events=[event_listener_js],
        id="click-image-clicks",
    ),
])

In [8]:
click_image_style = {
    "position": "absolute",
    "top": "50%",
    "left": "50%",
    "transform": "translate(-50%, -50%)",  # Offset by half the image dimensions to truly center it
    "width": "200px",
    # "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}


@app.callback(
    Output("click-image", "src"),
    Output("click-image", "style"),
    Output("figure", "clickData"),

    Input("figure", "clickData"),
    Input("figure-clicks", "event"),
    Input("click-image-clicks", "event"),
)
def toggle_click_image(clickData, figure_clicks, click_image_clicks):

    # When there's a click anywhere but on a data point
    if (figure_clicks or click_image_clicks) and not clickData:
        return "", {"display": "none"}, None

    if clickData:
        image_url = clickData["points"][0]["customdata"][0]
        return encode_image(image_url), click_image_style, None

    return "", {"display": "none"}, None

In [9]:
webbrowser.open('http://127.0.0.1:8050/')
app.run_server(port=8050, debug=True, use_reloader=False)
time.sleep(5)
app.run_server(port=8050, debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


# Hover & Click implementation

In [11]:
app = JupyterDash(__name__)

fig = px.scatter(
    df,
    x="x",
    y="y",
    hover_data={"t": True},
    custom_data=["image_urls"],
)

fig.update_traces(hovertemplate="%{customdata[1]}")

event_listener_js = {
    "event": "click",                 # Listen for click events
    "props": ["clientX", "clientY"],  # Capture the click's position on the screen
}

app.layout = html.Div([
    EventListener(
        dcc.Graph(id="figure", figure=fig, style={"height": "100vh", "width": "100vw"}),
        events=[event_listener_js],
        id="figure-clicks",
    ),
    html.Img(id="hover-image"),
    EventListener(
        html.Img(id="click-image", style={"display": "block"}),
        events=[event_listener_js],
        id="click-image-clicks",
    ),
    dcc.Store(id="click-image-visible", data=False)
])

In [12]:
hover_image_style = {
    "position": "absolute",
    "top": "82px",
    "right": "50px",
    "width": "200px",
    "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}


@app.callback(
    Output("hover-image", "src"),        # first output from the below function to hover-image (see app.layout)
    Output("hover-image", "style"),      # second output from the below function to hover-image (see app.layout)

    Input("figure", "hoverData"),  # input from the scatter-plot (see app.layout) to the below function
    Input("click-image-visible", "data")
)
def display_hover_image(hoverData, click_image_visible):
    if hoverData and not click_image_visible:
        image_url = hoverData["points"][0]["customdata"][0]
        return encode_image(image_url), hover_image_style
    return "", {"display": "none"}


click_image_style = {
    "position": "absolute",
    "top": "50%",
    "left": "50%",
    "transform": "translate(-50%, -50%)",
    "width": "600px",
    "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}


@app.callback(
    Output("click-image", "src"),
    Output("click-image", "style"),
    Output("figure", "clickData"),
    Output("click-image-visible", "data"),

    Input("figure", "clickData"),
    Input("figure-clicks", "event"),
    Input("click-image-clicks", "event"),
)
def toggle_click_image(clickData, figure_clicks, click_image_clicks):
    if (figure_clicks or click_image_clicks) and not clickData:
        return "", {"display": "none"}, None, False

    if clickData:
        image_url = clickData["points"][0]["customdata"][0]
        return encode_image(image_url), click_image_style, None, True

    return "", {"display": "none"}, None, False

In [13]:
webbrowser.open('http://127.0.0.1:8050/')
app.run_server(port=8050, debug=True, use_reloader=False)
time.sleep(5)
app.run_server(port=8050, debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


# All photos on a map

In [14]:
# Helper function to convert GPS coordinates to decimal
def convert_to_decimal(coord, ref):
    degrees, minutes, seconds = coord
    decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)
    if ref in ["S", "W"]:
        decimal = -decimal
    return decimal

In [15]:
# Extract GPS data using exifread
def extract_gps_data(image_path):
    try:
        with open(image_path, 'rb') as image_file:
            tags = exifread.process_file(image_file, stop_tag='GPS')

            timestamp = tags.get("Image DateTime")
            gps_latitude = tags.get("GPS GPSLatitude")
            gps_latitude_ref = tags.get("GPS GPSLatitudeRef")
            gps_longitude = tags.get("GPS GPSLongitude")
            gps_longitude_ref = tags.get("GPS GPSLongitudeRef")

            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = [float(x.num) / float(x.den) for x in gps_latitude.values]
                lon = [float(x.num) / float(x.den) for x in gps_longitude.values]

                lat_decimal = convert_to_decimal(lat, gps_latitude_ref.values)
                lon_decimal = convert_to_decimal(lon, gps_longitude_ref.values)

                return timestamp.values, lat_decimal, lon_decimal
        return timestamp.values, None, None
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None, None

In [16]:
# Process all images in a directory
def process_images(directory):
    data = []
    for filename in tqdm(os.listdir(directory)):
        if filename.lower().endswith(('.jpg', '.jpeg')):
            filepath = os.path.join(directory, filename)
            t, lat, lon = extract_gps_data(filepath)
            data.append({"filename": filepath, "timestamp": t, "latitude": lat, "longitude": lon})
    return data

In [17]:
metadata = process_images('./photos/')

100%|█████████████████████████████████████| 4420/4420 [00:01<00:00, 2356.91it/s]


In [18]:
df = pl.DataFrame(metadata).with_columns(pl.col('timestamp').str.to_datetime('%Y:%m:%d %H:%M:%S')).filter(pl.col('latitude').is_not_null())
df.head()

filename,timestamp,latitude,longitude
str,datetime[μs],f64,f64
"""./photos/PXL_20241015_02082151…",2024-10-14 22:08:21,40.73645,-73.986819
"""./photos/PXL_20240930_14080630…",2024-09-30 10:08:06,40.712542,-73.993772
"""./photos/PXL_20241014_22103181…",2024-10-14 18:10:31,40.753172,-73.978617
"""./photos/PXL_20241016_06274948…",2024-10-16 01:27:49,29.996244,-90.254758
"""./photos/PXL_20241017_15564377…",2024-10-17 10:56:43,29.960689,-90.057911


In [19]:
app = JupyterDash(__name__)

fig = px.density_mapbox(
    df.to_pandas(),
    lat="latitude",
    lon="longitude",
    hover_data={"timestamp": True},
    custom_data=["filename"],
    radius=10,
    center={"lat": 40.73, "lon": -73.97},
    mapbox_style="open-street-map",
    zoom=12,
)

fig.update_layout(margin={"l": 15, "r": 25, "t": 45, "b": 25}, coloraxis_showscale=False)
fig.update_traces(hovertemplate="%{customdata[1]}")

event_listener_js = {
    "event": "click",                 # Listen for click events
    "props": ["clientX", "clientY"],  # Capture the click's position on the screen
}

app.layout = html.Div([
    EventListener(
        dcc.Graph(id="figure", figure=fig, style={"height": "100vh", "width": "100vw"}),
        events=[event_listener_js],
        id="figure-clicks",
    ),
    html.Img(id="hover-image"),
    EventListener(
        html.Img(id="click-image", style={"display": "block"}),
        events=[event_listener_js],
        id="click-image-clicks",
    ),
    dcc.Store(id="click-image-visible", data=False)
])

In [20]:
hover_image_style = {
    "position": "absolute",
    "top": "82px",
    "right": "50px",
    "width": "200px",
    "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}


@app.callback(
    Output("hover-image", "src"),        # first output from the below function to hover-image (see app.layout)
    Output("hover-image", "style"),      # second output from the below function to hover-image (see app.layout)

    Input("figure", "hoverData"),  # input from the scatter-plot (see app.layout) to the below function
    Input("click-image-visible", "data")
)
def display_hover_image(hoverData, click_image_visible):
    if hoverData and not click_image_visible:
        image_url = hoverData["points"][0]["customdata"][0]
        return encode_image(image_url), hover_image_style
    return "", {"display": "none"}


click_image_style = {
    "position": "absolute",
    "top": "50%",
    "left": "50%",
    "transform": "translate(-50%, -50%)",
    "width": "600px",
    "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}


@app.callback(
    Output("click-image", "src"),
    Output("click-image", "style"),
    Output("figure", "clickData"),
    Output("click-image-visible", "data"),

    Input("figure", "clickData"),
    Input("figure-clicks", "event"),
    Input("click-image-clicks", "event"),
)
def toggle_click_image(clickData, figure_clicks, click_image_clicks):
    if (figure_clicks or click_image_clicks) and not clickData:
        return "", {"display": "none"}, None, False

    if clickData:
        image_url = clickData["points"][0]["customdata"][0]
        return encode_image(image_url), click_image_style, None, True

    return "", {"display": "none"}, None, False

In [21]:
webbrowser.open('http://127.0.0.1:8050/')
app.run_server(port=8050, debug=True, use_reloader=False)
time.sleep(5)
app.run_server(port=8050, debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
